In [1]:
import torch.nn as nn
import numpy as np
import gym

import ipynb.fs.full.misc as misc
import ipynb.fs.full.breeder as br
import ipynb.fs.full.mutation_functions as mf
import ipynb.fs.full.selection_functions as sf
import ipynb.fs.full.crossover_functions as cf
from ipynb.fs.full.agent_class import Agent
import ipynb.fs.full.net_class as nm

In [2]:
class WalkerAgentNet(nm.NetModule):
    def __init__(self):
        super(WalkerAgentNet, self).__init__()

        self.num_inputs  = 24
        self.num_outputs = 4

        #  define net structure here
        self.model = nn.Sequential(
            nn.Linear(self.num_inputs, 64),
            nn.Tanh(),
            nn.Linear(64, 64),
            nn.Tanh(),
            nn.Linear(64, 64),
            nn.Tanh(),
            nn.Linear(64, 64),
            nn.Tanh(),
            nn.Linear(64, 64),
            nn.Tanh(),
            nn.Linear(64, 64),
            nn.Tanh(),
            nn.Linear(64, 64),
            nn.Tanh(),
            nn.Linear(64, self.num_outputs),
            nn.Tanh(),
        )

        self.model = self.model.cuda()

        self.parameters_count = self.count_parameters()

    def forward(self, inputs):
        return self.model(inputs)

In [3]:
class WalkerAgent(Agent):
    def __init__(self, dna=None):
        super(WalkerAgent, self).__init__(dna, WalkerAgentNet())

    def step(self, observation):
        state = np.asarray(observation)

        torch_state = misc.single_to_torch(state, cuda=self.net.is_cuda())
        torch_action = self.net(torch_state)
        action = misc.single_from_torch(torch_action, cuda=self.net.is_cuda())

        self.add_experience(state, action)

        return action

    @staticmethod
    def produce_offspring(dna):
        return WalkerAgent(dna=dna)

In [4]:
env = gym.make("BipedalWalker-v2")

population_size = 50
population = [WalkerAgent()] * population_size

breeder = br.Breeder(population,
                     env,
                     300.,
                     generations=200,
                     elite_percent=0.1,
                     mutation_rate=0.1,
                     selection_pressure=0.2,
                     selection_func=sf.tournament,
                     crossover_func=cf.none,
                     mutation_func=mf.sm_r,
                     render=True)

agent = breeder.evolve()

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


KeyboardInterrupt: 

In [ ]:
s